# VacationPy


In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import requests
from pprint import pprint

# Import API key
from config_google import g_key

In [2]:
path = "weather_data_output.csv"

# Read the data 
weather_data = pd.read_csv(path)
weather_data

Country          City  Latitude (Degrees)  Longitude (Degrees)  \
0        TO         vaini            -21.2000            -175.2000   
1        AU     esperance            -33.8667             121.9000   
2        CL  punta arenas            -53.1500             -70.9167   
3        RU     saskylakh             71.9167             114.0833   
4        RU      makaryev             57.8797              43.8069   
..      ...           ...                 ...                  ...   
552      IN       mangrol             21.1167              70.1167   
553      MZ        maxixe            -23.8597              35.3472   
554      CI         tabou              4.4230              -7.3528   
555      PE      barranca            -10.7500             -77.7667   
556      CN       hanyang             32.1371             105.5126   

     Temperature (F)  Humidity (%)  Cloudiness (%)  Wind Speed (mph)  
0              84.20            79              75              4.61  
1              91.40            21              16             11.50  
2              50.00            66              90             20.71  
3             -20.72            85             100             14.14  
4              -7.22            90              85              9.01  
..               ...           ...             ...               ...  
552            75.36            58               0              4.59  
553            77.00           100              75             10.40  
554            77.97            90              38              6.33  
555            69.64            83               0              3.98  
556            45.00            45              99              5.99  

[557 rows x 8 columns]

In [3]:
#Test if data has humidity > 100
weather_data["Humidity (%)"].max()

100

### Humidity Heatmap

In [4]:
gmaps.configure(api_key=g_key)

In [6]:
locations1 = weather_data[["Latitude (Degrees)", "Longitude (Degrees)"]]
Humidity = weather_data["Humidity (%)"]

fig = gmaps.figure(center=[35,-100],zoom_level=5)

# Create heat layer
heat_layer = gmaps.heatmap_layer(
    locations1, 
    weights=Humidity, 
    dissipating=False, 
    max_intensity=10,
    point_radius=.5)

# Add layer
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria

In [7]:
#Ideal weather: 70 - 85 degrees Fahrenheit
temp = (weather_data["Temperature (F)"] > 75) & (weather_data["Temperature (F)"]<=85)
wind = weather_data["Wind Speed (mph)"]<=10.00
cloud = weather_data["Cloudiness (%)"]==0
humid_level = (weather_data["Humidity (%)"]>14) & (weather_data["Humidity (%)"]<=57)

ideal_weather_df = weather_data[temp & wind & cloud & humid_level]
ideal_weather_df

Country           City  Latitude (Degrees)  Longitude (Degrees)  \
27       HK       victoria             22.2855             114.1577   
373      LA  luang prabang             19.8856             102.1347   
415      SA       umm lajj             25.0213              37.2685   
511      IN        bikaner             28.0167              73.3000   
518      OM          nizwa             22.9333              57.5333   

     Temperature (F)  Humidity (%)  Cloudiness (%)  Wind Speed (mph)  
27             75.99            44               0              1.99  
373            80.60            47               0              2.30  
415            75.70            57               0              3.40  
511            81.97            17               0              7.76  
518            78.33            21               0              3.47

### Hotel Map

In [8]:
# Add a new column for hotel name and locations
ideal_weather_df["Hotel Name"]=" "
ideal_weather_df["Hotel Latitude"]= " "
ideal_weather_df["Hotel Longitude"]= " "

<ipython-input-8-fb1baa2a59e4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ideal_weather_df["Hotel Name"]=" "
<ipython-input-8-fb1baa2a59e4>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ideal_weather_df["Hotel Latitude"]= " "
<ipython-input-8-fb1baa2a59e4>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/

In [9]:

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "hotels",
    "keyword": "hotels",
    "key": g_key
}

for index, row in ideal_weather_df.iterrows():
    
    # get lat, lng from df
    lat = row["Latitude (Degrees)"]
    lng = row["Longitude (Degrees)"]
    
    # Add keyword location to params dictionary
    params["location"] = f"{lat},{lng}"

    # make request and print url
    print(f"Retrieving Results for Index {index}: {lat},{lng}")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    hotel_results = response['results']
    
    try:
        print(f"Closest hotel to {lat},{lng} is {hotel_results[0]['name']}.")
        
        ideal_weather_df.loc[index, 'Hotel Name'] = hotel_results[0]['name']
        ideal_weather_df.loc[index, 'Hotel Latitude'] = hotel_results[0]['geometry']['location']['lat']
        ideal_weather_df.loc[index, 'Hotel Longitude'] = hotel_results[0]['geometry']['location']['lng']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 27: 22.2855,114.1577
Closest hotel to 22.2855,114.1577 is The Landmark Mandarin Oriental, Hong Kong.
------------
Retrieving Results for Index 373: 19.8856,102.1347


C:\Users\jabuk\Anaconda3\envs\google\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


Closest hotel to 19.8856,102.1347 is Sofitel Luang Prabang.
------------
Retrieving Results for Index 415: 25.0213,37.2685
Closest hotel to 25.0213,37.2685 is HP Red Sea Hotel.
------------
Retrieving Results for Index 511: 28.0167,73.3
Closest hotel to 28.0167,73.3 is The Laxmi Niwas Palace.
------------
Retrieving Results for Index 518: 22.9333,57.5333
Closest hotel to 22.9333,57.5333 is Albayan Farm House.
------------


In [12]:
city_hotels_df = ideal_weather_df[["Hotel Name", "Country", "City", "Hotel Latitude", "Hotel Longitude"]]
city_hotels_df

Hotel Name Country           City  \
27   The Landmark Mandarin Oriental, Hong Kong      HK       victoria   
373                      Sofitel Luang Prabang      LA  luang prabang   
415                           HP Red Sea Hotel      SA       umm lajj   
511                     The Laxmi Niwas Palace      IN        bikaner   
518                         Albayan Farm House      OM          nizwa   

    Hotel Latitude Hotel Longitude  
27       22.280518      114.157939  
373      19.881046      102.135139  
415       25.03259       37.257006  
511      28.040114       73.331075  
518      22.922901       57.535144

In [13]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_details = [info_box_template.format(**row) for index, row in city_hotels_df.iterrows()]
locations = city_hotels_df[["Hotel Latitude", "Hotel Longitude"]]

In [14]:
# Add marker layer on top of heat map above
fig2= gmaps.figure(center=[28.040114,73.331075], zoom_level=4)

heat_layer = gmaps.heatmap_layer(
    locations1, 
    weights=Humidity,
    dissipating=False, 
    max_intensity=5,
    point_radius=.5)

marker_layer = gmaps.marker_layer(locations,
                                info_box_content=hotel_details,
                                display_info_box=None)

fig2.add_layer(heat_layer)
fig2.add_layer(marker_layer)

# Display figure
fig2

Figure(layout=FigureLayout(height='420px'))